## Import

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('../input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
!pip install -Uqq fastbook
import fastbook
fastbook.setup_book()
from fastbook import *

In [ ]:
## look into the dater
#from fastbook import *
#from kaggle import api
from pandas.api.types import is_string_dtype, is_numeric_dtype, is_categorical_dtype
from fastai.tabular.all import *
from sklearn.ensemble import RandomForestRegressor, RandomForestClassifier
from sklearn.tree import DecisionTreeRegressor, DecisionTreeClassifier
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
#!pip install dtreeviz

from IPython.display import Image, display_svg, SVG

In [ ]:
from sklearn.metrics import confusion_matrix, roc_auc_score, plot_confusion_matrix, precision_score, recall_score, classification_report, plot_roc_curve, roc_curve
from sklearn.model_selection import train_test_split

In [ ]:
import gc

In [ ]:
## pandas
pd.options.display.max_rows = 20
pd.options.display.max_columns = 12

In [ ]:
main_path = "../input/fraud-detection/"

In [ ]:
import xgboost as xgb
import random as r

In [ ]:
import joblib

## Load dater

In [ ]:
buildLv = False
buildSkf = True
FI=False

In [ ]:
%%time
dset_str="all"#all#50k#10k

to = load_pickle(main_path+"to_"+dset_str+"c.pkl")
to_tst = load_pickle(main_path+"to_tst_"+dset_str+"c.pkl")


In [ ]:
xs,y = to.train.xs,to.train.y
valid_xs,valid_y = to.valid.xs,to.valid.y
xs.shape

In [ ]:
xs.isna().sum()

In [ ]:
test_xs = to_tst.xs.copy()
test_xs.shape

In [ ]:
del to, to_tst; x=gc.collect()

## Functions

In [ ]:
def m_rep(m,xs,y): return classification_report(y, m.predict(xs), labels=[1,0], digits=4, output_dict=True)



def metrics(m, xs, y , valid_xs, valid_y): 
    tr_rep = m_rep(m,xs,y)
    vd_rep = m_rep(m,valid_xs,valid_y)
    tr_auc = roc_auc_score(y,m.predict_proba(xs)[:,1])
    oob_auc=0.0000
    #oob_auc = roc_auc_score(y,m.oob_decision_function_[:,1])
    vd_auc = roc_auc_score(valid_y,m.predict_proba(valid_xs)[:,1])
    print('{:.4f} ; {:.4f} ; {:.4f}'
          .format(tr_auc, oob_auc, vd_auc))

In [ ]:
## Plot  important features
def xgb_fi(m, df,df_real=None):
    fi = pd.DataFrame({'cols':df.columns, 'imp':m.feature_importances_}
                       ).sort_values('imp', ascending=False)
    #fi["isCont"] = fi.cols.isin(cont)
    fi["countNA"] = [df.loc[:,col].isna().sum()/len(df) if col in df.columns else float("NaN") for col in fi.cols]
    return fi

def plot_fi(fi):
    return fi.plot('cols', 'imp', 'barh', figsize=(12,7), legend=False)

In [ ]:
def format_test_preds(preds,test_xs,comment):
    #preds = m.predict_proba(test_xs[cols])[:,1]
    print(pd.DataFrame(preds).describe())
    
    ## Make into submission dataframe
    df_pred = pd.DataFrame({"TransactionID":test_xs.TransactionID.to_list(),
                            "isFraud": preds})
    
    ## save
    df_pred.to_csv(comment+"my_subs.csv",index=False)
    print(df_pred.shape)
    del df_pred; x=gc.collect()

In [ ]:
def xgb_lvd_fun(i):
    clf = xgb.XGBClassifier(n_estimators=1000, 
                        max_depth=8, #12
                        learning_rate= 0.05,
                        subsample= 0.6,#0.8 
                        colsample_bytree= 0.4, 
                        random_state = r.randint(0,9999),
#                         #USE CPU
#                         nthread=4)
                        # USE GPU
                        tree_method='gpu_hist')

    m = clf.fit(xs[cols].iloc[idxT], y.iloc[idxT], eval_set=[(xs[cols].iloc[idxV],y.iloc[idxV])],
                eval_metric= "auc", verbose=100, early_stopping_rounds=100)
    
    tr_auc = roc_auc_score(y.iloc[idxT], m.predict_proba(xs[cols].iloc[idxT])[:,1])
    vd_auc = roc_auc_score(y.iloc[idxV],m.predict_proba(xs[cols].iloc[idxV])[:,1])
    
    #tr_pred[idxT] += m.predict_proba(xs[cols].iloc[idxT])[:,1]/(skf.n_splits-1)
    #oof_pred[idxV] += m.predict_proba(xs[cols].iloc[idxV])[:,1]
    te_pred = m.predict_proba(test_xs[cols])[:,1]
    
    
        
    del m, clf; x=gc.collect()
    return [tr_auc, vd_auc], te_pred
    

In [ ]:
def xgb_pred_fun():
    from sklearn.model_selection import KFold
    skf = KFold(n_splits = 5, shuffle = False, random_state = 42)

    tr_pred = np.zeros(len(xs))
    oof_pred = np.zeros(len(xs))
    te_pred = np.zeros(len(test_xs))

    for i, (idxT, idxV) in enumerate(skf.split(xs,y)):
        print('Fold',i)
        print(' n_rows of train =',len(idxT),'rows of holdout =',min(idxV),"to",max(idxV))#len(idxV))
        clf = xgb.XGBClassifier(n_estimators=1000, 
                        max_depth=8, #12
                        learning_rate= 0.05,
                        subsample= 0.6,#0.8 
                        colsample_bytree= 0.4, 
                        random_state = r.randint(0,9999),
#                         #USE CPU
#                         nthread=4)
                        # USE GPU
                        tree_method='gpu_hist')

        m = clf.fit(xs[cols].iloc[idxT], y.iloc[idxT], eval_set=[(xs[cols].iloc[idxV],y.iloc[idxV])],
                eval_metric= "auc", verbose=100, early_stopping_rounds=100)

        tr_pred[idxT] += m.predict_proba(xs[cols].iloc[idxT])[:,1]/(skf.n_splits-1)
        oof_pred[idxV] += m.predict_proba(xs[cols].iloc[idxV])[:,1]
        te_pred += m.predict_proba(test_xs[cols])[:,1]/skf.n_splits
        del m; x=gc.collect()

    print('{:.4f} ; {:.4f} ; {:.4f}'
              .format(roc_auc_score(y,tr_pred),roc_auc_score(y,oof_pred),0.000))
    
    return te_pred

## Column selection due to EDA.

In [ ]:
## Create CENTS cols
def split_cols(df, col = "TransactionAmt"):
    df['cents'] = df[col].mod(1)
    df[col] = df[col].floordiv(1)
    
split_cols(xs), split_cols(test_xs)

In [ ]:
cols = xs.columns.to_list()

In [ ]:
## Vcol buckets (first filter based on correlation) From v11 EDA
v = []
v += ['V1', 'V3', 'V11', 'V9', 'V5', 'V7']
v += ['V13', 'V17', 'V24', 'V14', 'V20', 'V27', 'V34', 'V26', 'V30']
v += ['V36', 'V37', 'V47', 'V40', 'V48', 'V52', 'V41', 'V45']
v += ['V54', 'V65', 'V60', 'V67', 'V56', 'V68', 'V62', 'V55', 'V70']
v += ['V76', 'V89', 'V91', 'V81', 'V82', 'V87', 'V78', 'V88']
v += ['V127', 'V121', 'V99', 'V110', 'V104', 'V130', 'V129', 'V131', 'V109', 'V136', 'V116', 'V120', 'V125', 'V113', 'V118', 'V98', 'V107', 'V117', 'V115']
v += ['V138', 'V140', 'V142', 'V147', 'V155', 'V162']
v += ['V165', 'V160', 'V166']
v += ['V203', 'V207', 'V216', 'V187', 'V176', 'V173', 'V183', 'V215']
v += ['V169', 'V195', 'V201', 'V171', 'V174', 'V175', 'V209', 'V185', 'V188', 'V210', 'V198', 'V180']
v += ['V274', 'V264', 'V265', 'V261', 'V235', 'V223', 'V258', 'V260', 'V246', 'V252', 'V241', 'V266', 'V240', 'V277', 'V228', 'V226']
v += ['V220', 'V239', 'V271', 'V221', 'V234', 'V251']
v += ['V307', 'V291', 'V285', 'V290', 'V312', 'V297', 'V305', 'V320', 'V303', 'V286', 'V309', 'V284', 'V310']
v += ['V281', 'V301', 'V282', 'V315', 'V289', 'V296', 'V314', 'V283']
v += ['V332', 'V338', 'V337', 'V336', 'V325', 'V326', 'V328', 'V335']
len(v)


In [ ]:
def set_approach(a,b):
    return list(set(a)-set(b))

# Remove all v columns using below code
tf_V = [bool(re.search("^V"+"[0-9]+",col)) for col in xs.columns.to_list()]
cols = xs.columns[list(~np.array(tf_V))].to_list()
cols +=v
print(len(cols))


In [ ]:
#fi.cols[fi.imp>0.01].to_list()
cols_rem = ['TransactionID', 'TransactionDT']

In [ ]:
cols = set_approach(cols,cols_rem)

In [ ]:
cols

In [ ]:
len(cols)

## Local validation averging values based on randint

In [ ]:
## Linear split 80% 20%
msk = np.arange(len(xs))<0.8*len(xs) #usually 0.8, 0.8851 for the oversampling case
idxT = list(np.where(msk)[0])
idxV = list(np.where(~msk)[0])

In [ ]:
%%time
## Run LV 5 times

if buildLv:
    n=5
    preds = np.zeros(len(test_xs))
    auc_lst = []
    for i in range(n):
        print("iteation",i)
        a,temp = xgb_lvd_fun()
        preds += temp/n
        auc_lst.append(a)
        del temp; x=gc.collect()
        #print(a)
    auc_lst = np.array(auc_lst)
    print(auc_lst.mean(axis=0))
    print(auc_lst.std(axis=0))
    format_test_preds(preds,test_xs,"baselineLV")


In [ ]:
%%time
## Run 6 time kfold along with OOF prediction and fold based prediction of Test

if buildSkf:

    preds = xgb_pred_fun()
    format_test_preds(preds,test_xs,"baseline")
    del preds; x=gc.collect()

In [ ]:
## get feature importances

if FI==True:
    clf = xgb.XGBClassifier(n_estimators=1000, 
                        max_depth=8, #12
                        learning_rate= 0.05,
                        subsample= 0.6,#0.8 
                        colsample_bytree= 0.4, 
                        random_state = r.randint(0,9999),
    #                         #USE CPU
    #                         nthread=4)
                        # USE GPU
                        tree_method='gpu_hist')

    m = clf.fit(xs[cols].iloc[idxT], y.iloc[idxT], eval_set=[(xs[cols].iloc[idxV],y.iloc[idxV])],
                eval_metric= "auc", verbose=100, early_stopping_rounds=100)
    
    joblib.dump(m, 'm_lv.pkl')#save one m file for fi and other things...
    
    fi = xgb_fi(m, xs[cols])
    fi.to_csv("fi_lv.csv",index=False)
    print(fi.cols[fi.imp>0.0005].to_list())

In [ ]:
xs.shape